In [3]:
# install keras==2.3.1
# !pip install tensorflow==2.1.0
# !pip install keras_applications==1.0.8
# !pip install image-classifiers==1.0.0
# !pip install efficientnet==1.0.0

In [4]:
%matplotlib inline


import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import sys

#Our project root directory
PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname("__file__"), 
                                            os.pardir))
sys.path.append(PROJECT_ROOT)

from util.constants import DATA_PATHS
from util.dataset import OcelotDatasetLoader

import tensorflow as tf
import segmentation_models as sm
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)


Segmentation Models: using `keras` framework.


In [ ]:
# binary segmentation (this parameters are default when you call Unet('resnet34')
model = sm.Unet('resnet34', classes=1, activation='sigmoid')

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

# load your data
x_train, y_train, x_val, y_val = load_data(...)

# preprocess input
x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)

# define model
model = sm.Unet(BACKBONE, encoder_weights='imagenet')
model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
)

# fit model
# if you use data generator use model.fit_generator(...) instead of model.fit(...)
# more about `fit_generator` here: https://keras.io/models/sequential/#fit_generator
model.fit(
   x=x_train,
   y=y_train,
   batch_size=16,
   epochs=100,
   validation_data=(x_val, y_val),
)